# Model using Neural Network

## Import Libraries

In [20]:
import tensorflow as tf
import numpy as np
import pandas as pd
import seaborn as sns
from tensorflow.keras.layers import IntegerLookup
from tensorflow.keras.layers import Normalization
from tensorflow.keras.layers import StringLookup
import tensorflow.keras as keras

## Load The Data to DataFrame

In [68]:
df = pd.read_csv('cleaned_data.csv')

## Check the Columns' Data Type

In [69]:
df.columns

Index(['gender', 'age', 'workout_experience', 'workout_time', 'weight',
       'height', 'fitness_goal'],
      dtype='object')

In [70]:
# from sklearn.utils import shuffle


# shuffled_df = shuffle(df, random_state = 60)

## Split Data for Training and Validation


In [71]:
from sklearn.model_selection import train_test_split

train_df, val_df = train_test_split(shuffled_df, test_size = 0.2, random_state = 42)

## The training and validation set size

In [72]:
print('training : ',train_df.shape[0], " | validation : ", val_df.shape[0])

training :  723  | validation :  181



## Function Transform Dataframe into Tensor Dataset

In [73]:
def dataframe_transform(df, label):

    labels = df.pop(label)
    ds = tf.data.Dataset.from_tensor_slices((dict(df), labels))
    ds = ds.shuffle(buffer_size = len(df))

    return ds


## Transform Training and Validation Dataframe into Tensor Dataset

In [74]:

train_ds = dataframe_transform(train_df, 'fitness_goal')
val_ds   = dataframe_transform(val_df, 'fitness_goal')

## Batching the Dataset

In [75]:
train_ds = train_ds.batch(20)
val_ds   = val_ds.batch(20)

## Function to Encode Categorical Column

In [76]:
def encode_categorical_feature(column, name, dataset):

    lookup_class = IntegerLookup
    lookup = lookup_class(output_mode = "binary")

    column_ds = dataset.map(lambda x, y: x[name])
    column_ds = column_ds.map(lambda x: tf.expand_dims(x, -1))

    lookup.adapt(column_ds)
    encoded_column = lookup(column)

    return encoded_column

## Function to Encode Numerical Column

In [77]:
def encode_numerical_feature(column, name, dataset):

    normalizer = Normalization()
    column_ds = dataset.map(lambda x, y: x[name])
    column_ds = column_ds.map(lambda x: tf.expand_dims(x, -1))

    normalizer.adapt(column_ds)
    encoded_column = normalizer(column)

    return encoded_column

## Creating Tensor object from the column

In [78]:
gender = keras.Input(shape = (1,), name = 'gender', dtype = 'int64')
age    = keras.Input(shape = (1,), name = 'age')
workout_experience = keras.Input(shape = (1, ), name = 'workout_experience')
workout_time = keras.Input(shape = (1,), name = 'workout_time')
weight       = keras.Input(shape = (1,), name = 'weight')
height       = keras.Input(shape = (1,), name = 'height')

In [79]:
all_columns = [ gender, age, workout_experience, workout_time, height, weight ]

## Encode columns based on their data type

In [80]:
gender_encoded = encode_categorical_feature(gender, 'gender', train_ds)
age_encoded = encode_numerical_feature(age, 'age', train_ds)
workout_experience_encoded = encode_numerical_feature(workout_experience, 'workout_experience', train_ds)
workout_time_encoded = encode_numerical_feature(workout_time, 'workout_time', train_ds)
weight_encoded = encode_numerical_feature(weight, 'weight', train_ds)
height_encoded = encode_numerical_feature(height, 'height', train_ds)


In [81]:
all_features = keras.layers.concatenate([gender_encoded, age_encoded, workout_experience_encoded, workout_time_encoded, height_encoded, weight_encoded])

## Build the model architecture

In [94]:
x = keras.layers.Dense(30, activation = 'relu')(all_features)
x = keras.layers.Dense(20, activation = 'relu')(x)
x = keras.layers.Dense(10, activation = 'relu')(x)
x = keras.layers.Dropout(0.7)(x)

output = keras.layers.Dense(1, activation = 'sigmoid')(x)

In [95]:
model  = keras.Model(all_columns, output)
model.compile(optimizer = 'rmsprop', loss = 'binary_crossentropy', metrics = ['accuracy'])

## Train the model

In [96]:
from IPython.core.display import display, HTML

# Enable horizontal scrolling for output
display(HTML("<style>pre { white-space: pre !important; }</style>"))

# Generate a long output
long_output = "A" * 1000

model.fit(train_ds, epochs=300, validation_data = val_ds)

Epoch 1/300
37/37 [==============================] - 2s 10ms/step - loss: 0.7019 - accuracy: 0.5048 - val_loss: 0.6968 - val_accuracy: 0.4972
Epoch 2/300
37/37 [==============================] - 0s 3ms/step - loss: 0.6975 - accuracy: 0.5311 - val_loss: 0.6933 - val_accuracy: 0.5249
Epoch 3/300
37/37 [==============================] - 0s 3ms/step - loss: 0.6942 - accuracy: 0.5256 - val_loss: 0.6930 - val_accuracy: 0.4807
Epoch 4/300
37/37 [==============================] - 0s 3ms/step - loss: 0.6929 - accuracy: 0.5367 - val_loss: 0.6913 - val_accuracy: 0.5414
Epoch 5/300
37/37 [==============================] - 0s 3ms/step - loss: 0.6885 - accuracy: 0.5422 - val_loss: 0.6917 - val_accuracy: 0.5470
Epoch 6/300
37/37 [==============================] - 0s 3ms/step - loss: 0.6893 - accuracy: 0.5436 - val_loss: 0.6929 - val_accuracy: 0.5138
Epoch 7/300
37/37 [==============================] - 0s 3ms/step - loss: 0.6884 - accuracy: 0.5491 - val_loss: 0.6925 - val_accuracy: 0.5304
Epoch 8/300


Epoch 59/300
37/37 [==============================] - 0s 3ms/step - loss: 0.6539 - accuracy: 0.6003 - val_loss: 0.7086 - val_accuracy: 0.5083
Epoch 60/300
37/37 [==============================] - 0s 3ms/step - loss: 0.6596 - accuracy: 0.6072 - val_loss: 0.7150 - val_accuracy: 0.4972
Epoch 61/300
37/37 [==============================] - 0s 3ms/step - loss: 0.6572 - accuracy: 0.6017 - val_loss: 0.7083 - val_accuracy: 0.4917
Epoch 62/300
37/37 [==============================] - 0s 3ms/step - loss: 0.6535 - accuracy: 0.6127 - val_loss: 0.7091 - val_accuracy: 0.4862
Epoch 63/300
37/37 [==============================] - 0s 3ms/step - loss: 0.6602 - accuracy: 0.6086 - val_loss: 0.7119 - val_accuracy: 0.4917
Epoch 64/300
37/37 [==============================] - 0s 3ms/step - loss: 0.6484 - accuracy: 0.6086 - val_loss: 0.7172 - val_accuracy: 0.4751
Epoch 65/300
37/37 [==============================] - 0s 3ms/step - loss: 0.6570 - accuracy: 0.5906 - val_loss: 0.7150 - val_accuracy: 0.5083
Epoch 

37/37 [==============================] - 0s 3ms/step - loss: 0.6253 - accuracy: 0.6307 - val_loss: 0.7597 - val_accuracy: 0.4641
Epoch 117/300
37/37 [==============================] - 0s 3ms/step - loss: 0.6336 - accuracy: 0.6224 - val_loss: 0.7572 - val_accuracy: 0.4641
Epoch 118/300
37/37 [==============================] - 0s 3ms/step - loss: 0.6234 - accuracy: 0.6321 - val_loss: 0.7608 - val_accuracy: 0.4751
Epoch 119/300
37/37 [==============================] - 0s 3ms/step - loss: 0.6229 - accuracy: 0.6515 - val_loss: 0.7578 - val_accuracy: 0.4751
Epoch 120/300
37/37 [==============================] - 0s 3ms/step - loss: 0.6319 - accuracy: 0.6321 - val_loss: 0.7488 - val_accuracy: 0.4807
Epoch 121/300
37/37 [==============================] - 0s 3ms/step - loss: 0.6227 - accuracy: 0.6293 - val_loss: 0.7504 - val_accuracy: 0.4917
Epoch 122/300
37/37 [==============================] - 0s 3ms/step - loss: 0.6070 - accuracy: 0.6653 - val_loss: 0.7678 - val_accuracy: 0.4586
Epoch 123/300

37/37 [==============================] - 0s 3ms/step - loss: 0.6045 - accuracy: 0.6639 - val_loss: 0.8287 - val_accuracy: 0.4586
Epoch 174/300
37/37 [==============================] - 0s 3ms/step - loss: 0.5855 - accuracy: 0.6625 - val_loss: 0.8423 - val_accuracy: 0.4586
Epoch 175/300
37/37 [==============================] - 0s 3ms/step - loss: 0.5793 - accuracy: 0.6653 - val_loss: 0.8511 - val_accuracy: 0.4586
Epoch 176/300
37/37 [==============================] - 0s 3ms/step - loss: 0.5836 - accuracy: 0.6708 - val_loss: 0.8506 - val_accuracy: 0.4365
Epoch 177/300
37/37 [==============================] - 0s 3ms/step - loss: 0.5833 - accuracy: 0.6653 - val_loss: 0.8400 - val_accuracy: 0.4475
Epoch 178/300
37/37 [==============================] - 0s 3ms/step - loss: 0.5808 - accuracy: 0.6625 - val_loss: 0.8392 - val_accuracy: 0.4530
Epoch 179/300
37/37 [==============================] - 0s 3ms/step - loss: 0.5890 - accuracy: 0.6584 - val_loss: 0.8391 - val_accuracy: 0.4586
Epoch 180/300

37/37 [==============================] - 0s 3ms/step - loss: 0.5828 - accuracy: 0.6598 - val_loss: 0.8840 - val_accuracy: 0.4696
Epoch 231/300
37/37 [==============================] - 0s 3ms/step - loss: 0.5307 - accuracy: 0.7082 - val_loss: 0.9238 - val_accuracy: 0.4530
Epoch 232/300
37/37 [==============================] - 0s 3ms/step - loss: 0.5489 - accuracy: 0.6943 - val_loss: 0.9260 - val_accuracy: 0.4475
Epoch 233/300
37/37 [==============================] - 0s 3ms/step - loss: 0.5628 - accuracy: 0.6528 - val_loss: 0.9253 - val_accuracy: 0.4365
Epoch 234/300
37/37 [==============================] - 0s 3ms/step - loss: 0.5585 - accuracy: 0.6916 - val_loss: 0.9104 - val_accuracy: 0.4586
Epoch 235/300
37/37 [==============================] - 0s 3ms/step - loss: 0.5621 - accuracy: 0.6888 - val_loss: 0.9141 - val_accuracy: 0.4586
Epoch 236/300
37/37 [==============================] - 0s 3ms/step - loss: 0.5406 - accuracy: 0.6846 - val_loss: 0.9377 - val_accuracy: 0.4475
Epoch 237/300

37/37 [==============================] - 0s 3ms/step - loss: 0.5076 - accuracy: 0.7220 - val_loss: 1.0208 - val_accuracy: 0.4530
Epoch 288/300
37/37 [==============================] - 0s 3ms/step - loss: 0.5331 - accuracy: 0.6805 - val_loss: 1.0225 - val_accuracy: 0.4641
Epoch 289/300
37/37 [==============================] - 0s 3ms/step - loss: 0.5277 - accuracy: 0.7026 - val_loss: 1.0287 - val_accuracy: 0.4696
Epoch 290/300
37/37 [==============================] - 0s 3ms/step - loss: 0.5269 - accuracy: 0.6846 - val_loss: 1.0036 - val_accuracy: 0.4586
Epoch 291/300
37/37 [==============================] - 0s 3ms/step - loss: 0.5400 - accuracy: 0.6791 - val_loss: 1.0084 - val_accuracy: 0.4475
Epoch 292/300
37/37 [==============================] - 0s 3ms/step - loss: 0.5179 - accuracy: 0.7068 - val_loss: 1.0109 - val_accuracy: 0.4641
Epoch 293/300
37/37 [==============================] - 0s 3ms/step - loss: 0.5238 - accuracy: 0.6833 - val_loss: 1.0278 - val_accuracy: 0.4530
Epoch 294/300

In [ ]:
df.info()

In [69]:
input_data = {
    'gender' : 1,
    'age'    :21,
    'workout_experience' : 0,
    'workout_time' : 60,
    'weight' : 65,
    'height' : 170,
}

In [70]:
input_dict = {}

In [71]:
for feature, value in input_data.items() :
    input_dict[feature] = tf.convert_to_tensor([value])

In [72]:
input_dict

{'gender': <tf.Tensor: shape=(1,), dtype=int32, numpy=array([1])>,
 'age': <tf.Tensor: shape=(1,), dtype=int32, numpy=array([21])>,
 'workout_experience': <tf.Tensor: shape=(1,), dtype=int32, numpy=array([0])>,
 'workout_time': <tf.Tensor: shape=(1,), dtype=int32, numpy=array([60])>,
 'weight': <tf.Tensor: shape=(1,), dtype=int32, numpy=array([65])>,
 'height': <tf.Tensor: shape=(1,), dtype=int32, numpy=array([170])>}

In [76]:
predictions = model.predict(input_dict)

1/1 [==============================] - 0s 34ms/step


In [77]:
predictions

array([[0.5033811]], dtype=float32)

In [78]:
if predictions[0][0] >= 0.5 :
    print("You should do muscle up !")
else :
    print("You should do weight loss !")

You should do muscle up !


In [21]:
import joblib

# Save the trained model to a file
joblib.dump(model, 'model_1_v2.pkl')

INFO:tensorflow:Assets written to: ram://a02c0d99-312e-4275-b03e-2019695547c0/assets


['model_1_v2.pkl']